In [1]:
from pyspark.sql import SparkSession
# Inicializa uma sessão do Spark com as configurações do Iceberg

spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config("spark.driver.host", "localhost") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .config("spark.sql.catalog.local.create-namespace", "true") \
  .config("spark.driver.bindAddress", "localhost") \
  .getOrCreate()

25/04/21 23:41:41 WARN Utils: Your hostname, gabriel-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/21 23:41:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/gabriel/Documents/GitHub/eg-dados-apache-spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gabriel/.ivy2/cache
The jars for the packages stored in: /home/gabriel/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-51d59057-c7de-4382-a01b-f3713d773d2e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
:: resolution report :: resolve 368ms :: artifacts dl 10ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache

In [3]:
# Criação da tabela
spark.sql(
    """
    CREATE TABLE IF NOT EXISTS local.iceberg.covid_daily_vaccinations (
        date DATE,
        location STRING,
        daily_vaccinations DOUBLE,
        people_fully_vaccinated DOUBLE
    ) USING iceberg PARTITIONED BY (year(date));
    """
)

DataFrame[]

In [4]:
spark.sql("SELECT * FROM local.iceberg.covid_daily_vaccinations").show()

+----------+--------+------------------+-----------------------+
|      date|location|daily_vaccinations|people_fully_vaccinated|
+----------+--------+------------------+-----------------------+
|2024-01-01|   Aruba|               0.0|                   NULL|
|2024-01-02|   Aruba|               0.0|                   NULL|
|2024-01-03|   Aruba|               0.0|                   NULL|
|2024-01-04|   Aruba|               0.0|                   NULL|
|2024-01-05|   Aruba|               0.0|                   NULL|
|2024-01-06|   Aruba|               0.0|                   NULL|
|2024-01-07|   Aruba|               0.0|                   NULL|
|2024-01-08|   Aruba|               0.0|                84364.0|
|2024-01-09|   Aruba|               0.0|                   NULL|
|2024-01-10|   Aruba|               0.0|                84364.0|
|2024-01-11|   Aruba|               1.0|                   NULL|
|2024-01-12|   Aruba|               1.0|                84365.0|
|2024-01-13|   Aruba|    

In [5]:
from pyspark.sql.functions import to_date, col

df = spark.read.csv("data/covid-19/vaccinations.csv", header=True)

df = df.withColumn("date", to_date(df["date"], "yyyy-MM-dd"))
df = df.withColumn("daily_vaccinations", col("daily_vaccinations").cast("double"))
df = df.withColumn("people_fully_vaccinated", col("people_fully_vaccinated").cast("double"))

df.writeTo("local.iceberg.covid_daily_vaccinations").append()

In [17]:
spark.sql(
    """
    INSERT INTO local.iceberg.covid_daily_vaccinations
    VALUES
        (DATE('2025-04-21'), 'Brazil', 20000.0, 15000000.0),
        (DATE('2025-04-20'), 'USA', 30000.0, 25000000.0),
        (DATE('2025-04-19'), 'India', 40000.0, 35000000.0)
    """
)

DataFrame[]

In [18]:
spark.sql(
    """ 
    UPDATE local.iceberg.covid_daily_vaccinations
    SET daily_vaccinations = 50000.0
    WHERE location = 'Brazil'
    AND date = DATE('2025-04-21')
    """
)


DataFrame[]

In [19]:
spark.sql(
    """
    SELECT * FROM local.iceberg.covid_daily_vaccinations 
    WHERE location = 'Brazil' AND date = DATE('2025-04-21')
    """
).show()

+----------+--------+------------------+-----------------------+
|      date|location|daily_vaccinations|people_fully_vaccinated|
+----------+--------+------------------+-----------------------+
|2025-04-21|  Brazil|           50000.0|                  1.5E7|
+----------+--------+------------------+-----------------------+



In [20]:
spark.sql(
    """
    DELETE FROM local.iceberg.covid_daily_vaccinations
    WHERE location = 'Brazil' AND date = DATE('2025-04-21')
    """
).show()

++
||
++
++

